In [1]:
!pip install transformers
!pip install sentencepiece
!pip install rouge_score
!pip install accelerate -U
!pip install transformers[torch]
!pip install accelerate -U
!pip install accelerate>=0.20.1
!pip install meteor
!pip install --upgrade google-cloud-firestore
!pip install firebase_admin
!pip install wandb

  Obtaining dependency information for google-cloud-firestore from https://files.pythonhosted.org/packages/36/3c/ff2a16dc5b56afdbf41993f7b1d3e1d227f0fe030eb57675a4f38cf83298/google_cloud_firestore-2.13.1-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.5/289.5 kB 7.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-cloud-firestore
    Found existing installation: google-cloud-firestore 2.13.0
    Uninstalling google-cloud-firestore-2.13.0:
      Successfully uninstalled google-cloud-firestore-2.13.0


In [1]:
import tensorflow as tf
tf.__version__

'2.14.0'

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available.")

In [ ]:
!nvidia-smi

In [14]:
import os
import nltk
import wandb
import torch
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import firestore
from transformers import MarianTokenizer
import nltk.translate.meteor_score as meteor
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split
from transformers import (AutoModel, MarianMTModel, MarianTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, PreTrainedTokenizerBase, PreTrainedModel, T5ForConditionalGeneration)

In [15]:
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
!sudo chmod 777 /home/mac/.config/gcloud/application_default_credentials.json
#!sudo chmod 777 /home/mac/marian-MT/yoruba_tokenizer/tokenizer_config.json

In [3]:
# Initialize Firestore
db = firestore.Client()

# Firestore collection name
collection_name = 'text_data'  # Replace with your collection name

# Reference to the Firestore collection
collection_ref = db.collection(collection_name)

# Query Firestore to get all documents
documents = collection_ref.stream()
data = []
# Iterate over the documents and retrieve data
for document in documents:
    document_data = document.to_dict()
    data.append(document_data)

In [1]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [4]:

"""df = pd.DataFrame(data)"""

In [5]:
"""df.head()"""

,English,Yoruba,ID
0,"Given all that, here are some questions you ca...","Pẹ̀lú gbogbo ìwọ̀nyí, ìwọ̀nyí ni ìbéèrè tí o l...",36308
1,Then God said: Let the earth bring forth livin...,"Lẹ́yìn náà, Ọlọ́run wá sọ pé: Kí ilẹ̀ mú àwọn ...",21439
2,You can say some of this is education.,Ẹ lè sọ wí pé díẹ̀ nínú àwọn yìí jẹ́ ètò-ẹ̀kọ́.,1159
3,My whole being will rejoice in my God.,Gbogbo ara mi máa yọ̀ nínú Ọlọ́run mi.,19428
4,You heard that it was said to those of ancient...,Ẹ gbọ́ pé a sọ fún àwọn èèyàn àtijọ́ pé: O ò g...,31445


In [6]:
"""df.to_parquet('MT-data.parquet')"""

In [20]:
df = pd.read_parquet("MT-data.parquet")

In [7]:
# Assuming the columns are named 'English', 'Yoruba', and 'ID'
english_texts = df['English'].tolist()
yoruba_texts = df['Yoruba'].tolist()

In [17]:
# Load a pretrained SentencePiece model
model_name = "Helsinki-NLP/opus-mt-yo-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)

# Load pre-trained model to extract embeddings
source_embedder = AutoModel.from_pretrained('bert-base-uncased')
target_embedder = AutoModel.from_pretrained('bert-base-uncased')

In [114]:
yoruba_tokenized = tokenizer(yoruba_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
english_tokenized = tokenizer(english_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [8]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-yo-en', vocab_size=56836, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	56835: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [112]:
# Split data into training and evaluation sets
yoruba_train, yoruba_eval, english_train, english_eval = train_test_split(yoruba_texts, english_texts, test_size=0.1, random_state=42)

In [116]:
!gsutil cp -r tokenizer gs://nkenne-machinelearning-bucket/marian-MT/yoruba_tokenizer

Copying file://tokenizer/vocab.json [Content-Type=application/json]...
Copying file://tokenizer/tokenizer_config.json [Content-Type=application/json]...
Copying file://tokenizer/target.spm [Content-Type=application/octet-stream]...  
Copying file://tokenizer/special_tokens_map.json [Content-Type=application/json]...
- [4 files][  2.2 MiB/  2.2 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://tokenizer/source.spm [Content-Type=application/octet-stream]...
- [5 files][  3.0 MiB/  3.0 MiB]                                                
Operation completed over 5 objects/3.0 MiB.                                      


In [51]:
!pip install google-cloud-storage --upgrade

  Obtaining dependency information for google-cloud-storage from https://files.pythonhosted.org/packages/04/72/71b1b531cefa1daff8f6a2a70b4d4fa18dd4da851b5486d53578811b0838/google_cloud_storage-2.13.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth<3.0dev,>=2.23.3 from https://files.pythonhosted.org/packages/86/a7/75911c13a242735d5aeaca6a272da380335ff4ba5f26d6b2ae20ff682d13/google_auth-2.23.4-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.3/183.3 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.23.0
    Uninstalling google-auth-2.23.0:
      Successfully uninstalled google-auth-2.23.0
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '_metadata.cpython-310.pyc'
Consider using the `--user` option or check the permissions.



In [10]:
yoruba_tokenized = tokenizer(yoruba_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [117]:
yoruba_tokenizer = MarianTokenizer.from_pretrained("gs://nkenne-machinelearning-bucket/marian-MT/yoruba_tokenizer")

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'gs://nkenne-machinelearning-bucket/marian-MT/yoruba_tokenizer'. Use `repo_type` argument if needed.

In [60]:
yoruba_tokenizer

MarianTokenizer(name_or_path='gs://nkenne-machinelearning-bucket/marian-MT/yoruba_tokenizer', vocab_size=56836, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	56835: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [61]:
yoruba_tokenized = yoruba_tokenizer(yoruba_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
english_tokenized = yoruba_tokenizer(english_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [63]:
class TranslationDataset(Dataset):
    def __init__(self, source_texts, target_texts, source_tokenizer, target_tokenizer, source_embedder, target_embedder):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.source_tokenizer = source_tokenizer
        self.target_tokenizer = target_tokenizer
        self.target_embedder = target_embedder
        self.source_embedder = source_embedder 

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source_text = self.source_texts[idx]
        target_text = self.target_texts[idx]

        source_inputs = self.source_tokenizer(source_text, return_tensors="pt", padding="max_length", max_length=512, truncation=True)
        target_inputs = self.target_tokenizer(target_text, return_tensors="pt", padding="max_length", max_length=512, truncation=True)

        # Extract embeddings for the source and target text using the respective embedder
        # You can use a pre-trained transformer model or any other method here
        source_embeddings = self.source_embedder(source_tokens["input_ids"]).last_hidden_state.squeeze(0)
        target_embeddings = self.target_embedder(target_tokens["input_ids"]).last_hidden_state.squeeze(0)


        # return {
        #     "input_ids": source_inputs["input_ids"].flatten(),
        #     "attention_mask": source_inputs["attention_mask"].flatten(),
        #     "labels": target_inputs["input_ids"].flatten(),
        # }

        return {
            "source_embeddings": source_embeddings,  # Embeddings for the source language
            "target_embeddings": target_embeddings,  # Embeddings for the target language
            "labels": target_tokens["input_ids"].squeeze(0),  # Target tokens as labels
        }

translation_dataset = TranslationDataset(yoruba_train, english_train, yoruba_tokenized, english_tokenized, source_embedder, target_embedder)
evaluation_dataset = TranslationDataset(yoruba_eval, english_eval, yoruba_tokenized, english_tokenized, source_embedder, target_embedder)
data_loader = DataLoader(translation_dataset, batch_size=4, shuffle=True)
eval_data_loader = DataLoader(evaluation_dataset, batch_size=4, shuffle=False)

In [ ]:
'''
NEW
'''
class CustomT5ForConditionalGeneration(T5ForConditionalGeneration):
    def forward(self, input_ids=None, attention_mask=None, decoder_input_ids=None, labels=None, encoder_outputs=None, **kwargs):
        if input_ids is None:
            # Use precomputed embeddings instead of token IDs
            encoder_outputs = self.encoder(inputs_embeds=input_ids, attention_mask=attention_mask, **kwargs)
        else:
            # Fallback to normal behavior if input_ids are provided
            encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
        
        # Decoder logic remains unchanged
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=encoder_outputs[0],
            attention_mask=attention_mask,
            labels=labels,
            **kwargs
        )
        
        return decoder_outputs



In [64]:
model = MarianMTModel.from_pretrained(model_name)
model_2 =  CustomT5ForConditionalGeneration.from_pretrained('t5-small') '''Modified''' # 't5-base' 

In [71]:
# Define GCS bucket and local directory
bucket_name = "nkenne-machinelearning-bucket"
local_model_directory = "model"  # Replace with your local directory path
gcs_path = "marian-MT/model"  # Replace with your GCS bucket path

In [81]:
'''
Modified
'''

training_args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=8,  # Set this to a larger value
    gradient_accumulation_steps=4,
    evaluation_strategy="epoch",
    save_total_limit=3,
    output_dir="model",
    num_train_epochs=1,  # Increase if needed
    save_steps=400,
    weight_decay=0.01,  # For reqularization
    eval_steps=400,
    save_steps=500
    logging_steps=100,
    learning_rate=1e-5,  # Experiment with learning rate schedule
    overwrite_output_dir=True,
    load_best_model_at_end=True,  # Load the best model at the end of training
    fp16=True
)

In [82]:
import threading
import time
from google.cloud import storage

def sync_to_gcs(local_directory, gcs_path):
    storage_client = storage.Client()
    bucket_name = "nkenne-machinelearning-bucket"

    while True:
        time.sleep(300)  # Adjust the time interval for synchronization as needed (e.g., sync every 5 minutes)

        bucket = storage_client.get_bucket(bucket_name)
        blob = bucket.blob(gcs_path)

        # Copy contents of local directory to GCS
        blob.upload_from_filename(local_directory, if_generation_match=None)


In [83]:
'''
Modified
'''

# Initialize data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=english_tokenized,
    model=model_2,
)

In [84]:
df.tail()

,English,Yoruba,ID
53317,They were singing the song of Moses the slave ...,Wọ́n ń kọ orin Mósè ẹrú Ọlọ́run àti orin Ọ̀dọ́...,25454
53318,"I graduated high school in Cleveland, Ohio, 1975.",Mo kẹ́kọ̀ọ́ jáde ní ilé-ẹ̀kọ́ mẹ́wàá Cleveland...,41099
53319,It is simply a euphemism for theft to say àfọw...,Àdàpè olè ní ń jẹ́ àfọwọ́rá.,40469
53320,"And apart from that, she's not just a Nigerian.","Yàtọ̀ sí èyí, kì í ṣe ọmọ orílẹ̀-èdè Nàìjíríà ...",37029
53321,Frankly speaking.,Ní òtítọ́ sísọ.,8960


In [85]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(56836, 512, padding_idx=56835)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(56836, 512, padding_idx=56835)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,),

In [86]:
print("Cuda available: ", torch.cuda.is_available())

Cuda available:  True


In [42]:
# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model_2,
    data_collator=data_collator,
    args=training_args,
    train_dataset=translation_dataset,
    eval_dataset=evaluation_dataset,
    #compute_metrics=compute_metrics,
)
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
'''
NEW
'''

hypotheses = []

# Iterate over the test data
for line in test_data_yor:
    # Tokenize the Yoruba sentence and convert it to embeddings
    embeddings = yoruba_tokenizer(line, return_tensors="pt", padding=True, truncation=True, max_length=512)["input_ids"].to(device)
    
    # Forward pass through the model using embeddings instead of token IDs. Assuming the model has been modified to accept embeddings
    encoder_outputs = checkpoint_model.encoder(inputs_embeds=embeddings)  # Use the encoder with embeddings
    
    # Generate translations by decoding the output embeddings from the decoder
    # You may want to use the decoder with its own logic for generation
    translated_embeddings = checkpoint_model.decoder.generate(
        encoder_outputs=encoder_outputs[0],  # This is the encoded embeddings from the encoder
        max_length=512  # You can set your max length for generation
    )
    
    # Decode the translated embeddings into tokens and then into text
    translated_text = english_tokenizer.decode(translated_embeddings[0], skip_special_tokens=True)
    
    # Append the translated text to hypotheses
    hypotheses.append(translated_text)


# Calculate the METEOR score
meteor_scores = []
for i in range(len(hypotheses)):
    # Tokenize the reference and hypothesis
    reference_tokens = references[i].split()
    hypothesis_tokens = hypotheses[i].split()

    # Calculate METEOR score
    meteor_score = meteor.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score)

# Print the results
for i in range(len(hypotheses)):
    print("Input sentence:", test_data_yor[i])
    print("Predicted translation:", hypotheses[i])
    print("Actual translation:", references[i])
    print(f"METEOR score: {meteor_scores[i]:.4f}")
    print()

In [21]:
# Read your Yoruba test data
with open("small_vocab_yor", "r") as f:
    test_data_yor = f.readlines()

# Translate the Yoruba sentences and store them as hypotheses
hypotheses = []
for line in test_data_yor:
    input_ids = yoruba_tokenizer(line, return_tensors="pt")["input_ids"].to(device)
    translated_text = checkpoint_model.generate(input_ids)[0]
    translated_text = english_tokenizer.decode(translated_text)
    hypotheses.append(translated_text)

# Read the English references
with open("small_vocab_en", "r") as f:
    references = f.readlines()

# Calculate the METEOR score
meteor_scores = []
for i in range(len(hypotheses)):
    # Tokenize the reference and hypothesis
    reference_tokens = references[i].split()
    hypothesis_tokens = hypotheses[i].split()

    # Calculate METEOR score
    meteor_score = meteor.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score)

# Print the results
for i in range(len(hypotheses)):
    print("Input sentence:", test_data_yor[i])
    print("Predicted translation:", hypotheses[i])
    print("Actual translation:", references[i])
    print(f"METEOR score: {meteor_scores[i]:.4f}")
    print()

# Calculate the overall METEOR score
overall_meteor_score = sum(meteor_scores) / len(meteor_scores)
print(f"Overall METEOR score: {overall_meteor_score:.4f}")

Input sentence: Ó sése kí n rin  ìrìn àjò lọ sí ibi tó farasin ní oṣù January.

Predicted translation: <pad> He started me to travel to the place that was in the month of January.</s>
Actual translation: I might travel out to a quiet place in the month of January. .

METEOR score: 0.4470

Input sentence: Mo fẹ́ràn láti rẹjú ní wàrawàra lẹ̀yìn oúnjẹ ò̩sán lojojúmọ́.

Predicted translation: <pad> I love to be carried before the evening of the night.</s>
Actual translation: I like to take a nap immediately after lunch everyday.

METEOR score: 0.0980

Input sentence: Mo kórìíra láti máa lo sí àpèjẹ pèlú àwọn ọ̀rẹ́ mi ṣùgbọ́n mo fẹ́ràn láti máa wo tẹlifíṣọ̀n

Predicted translation: <pad> I hate to use my friends with my friends but I love to look at movies</s>
Actual translation: I  don’t like going out to parties with friends but I like watching TV..

METEOR score: 0.3574

Input sentence: Ẹ̀rù bà mí nígbà tí mo rí fọ́tò ẹnì kan tó wọ aṣọ ológun ní ọ̀dẹ̀dẹ̀ ilé náà.

Predi

TRAIN FROM CHECKPOINT

In [33]:
# Load a pretrained SentencePiece model
model_name = "Helsinki-NLP/opus-mt-yo-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [37]:
run = wandb.init(project="Yor-En-MachineTranslation", job_type="train", settings=wandb.Settings(start_method="fork"))
# Set the WANDB_STORAGE environment variable to your GCS bucket URL
!export WANDB_STORAGE="gs://nkenne-machinelearning-bucket/marian-MT/model"
os.environ["WANDB_LOG_MODEL"]="checkpoint"

def batch_encode(texts, tokenizer, max_length=512):
    encoded_texts = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    input_ids = encoded_texts["input_ids"].flatten()
    attention_mask = encoded_texts["attention_mask"].flatten()
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
    }

class TranslationDataset(Dataset):
    def __init__(self, source_texts, target_texts, tokenizer, max_length=512):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source_text = self.source_texts[idx]
        target_text = self.target_texts[idx]

        encoded_sources = batch_encode([source_text], self.tokenizer, self.max_length)
        encoded_targets = batch_encode([target_text], self.tokenizer, self.max_length)

        return {
            "input_ids": encoded_sources["input_ids"],
            "attention_mask": encoded_sources["attention_mask"],
            "labels": encoded_targets["input_ids"],
        }

if __name__ == "__main__":
    # Load data
    df = pd.read_parquet("MT-data.parquet")
    english_texts = df['English'].tolist()
    yoruba_texts = df['Yoruba'].tolist()

    # Split data into training and evaluation sets
    yoruba_train, yoruba_eval, english_train, english_eval = train_test_split(yoruba_texts, english_texts, test_size=0.1, random_state=42)

    # Prepare training and evaluation datasets
    batch_size = 16
    train_dataset = TranslationDataset(yoruba_train, english_train, tokenizer)
    eval_dataset = TranslationDataset(yoruba_eval, english_eval, tokenizer)

   #Load checkpoint
    #model = wandb.restore("kingalagbe/MachineTranslation-project/checkpoint-jcshp99a")
    # Load a pretrained SentencePiece model
    model = MarianMTModel.from_pretrained("model/checkpoint-22400")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Prepare data collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
    )

    # Set training arguments
    training_args = Seq2SeqTrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        evaluation_strategy="steps",
        save_total_limit=2,
        output_dir="model",
        num_train_epochs=15,
        save_steps=400,
        eval_steps=400,
        logging_steps=100,
        learning_rate=1e-5,
        overwrite_output_dir=True,
        load_best_model_at_end=True,
        report_to="wandb"
    )

    # Create trainer
    trainer = Seq2SeqTrainer(
        model=model,
        data_collator=data_collator,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    # Train the model
    trainer.train()

eval/loss,▃▄▅▆▇▇▇▇██▇██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁
eval/runtime,▆▄▄▆▄▄▃▃▄▃▄▃▂▂▂▅▅▂▄▁▂▂▁▁▁▁▁▅▃▃▂▂▁▃▃▄▅▂▂█
eval/samples_per_second,▃▅▅▃▅▅▆▆▅▆▅▆▇▇▇▄▄▇▅█▇▇█████▄▆▆▇▇█▆▆▅▄▇▇▁
eval/steps_per_second,▃▅▅▃▅▅▆▆▅▆▅▆▇▇▇▄▄▇▅█▇▇█████▄▆▆▇▇█▆▆▅▄▇▇▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▁▂▁▁▂▂▁▂▁▂▂▂▂▂▃▃▃▄▄▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇█▇
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


Step,Training Loss,Validation Loss
400,0.238700,1.210559
800,0.248500,1.222804
1200,0.239100,1.229103
1600,0.227900,1.236547
2000,0.246700,1.240833
2400,0.249900,1.245431
2800,0.248600,1.245172
3200,0.245300,1.246650
3600,0.241700,1.249716
4000,0.255100,1.249992


wandb: Adding directory to artifact (./model/checkpoint-400)... Done. 7.2s
wandb: WARNING Failed to cache /home/jupyter/.local/share/wandb/artifacts/staging/tmpibdz2sw4, ignoring [Errno 2] No such file or directory: '/home/jupyter/.cache/wandb/artifacts'
wandb: WARNING Failed to cache /home/jupyter/.local/share/wandb/artifacts/staging/tmpiqpfkjzx, ignoring [Errno 2] No such file or directory: '/home/jupyter/.cache/wandb/artifacts'
wandb: WARNING Failed to cache /home/jupyter/.local/share/wandb/artifacts/staging/tmprxh6vtjb, ignoring [Errno 2] No such file or directory: '/home/jupyter/.cache/wandb/artifacts'
wandb: WARNING Failed to cache /home/jupyter/.local/share/wandb/artifacts/staging/tmplvy034hn, ignoring [Errno 2] No such file or directory: '/home/jupyter/.cache/wandb/artifacts'
wandb: WARNING Failed to cache /home/jupyter/.local/share/wandb/artifacts/staging/tmpb2_3a8iq, ignoring [Errno 2] No such file or directory: '/home/jupyter/.cache/wandb/artifacts'
wandb: WARNING Failed to 

KeyboardInterrupt: 

In [16]:
artifact = wandb.Artifact('MT_model', type='model')
artifact.add_reference('gs://nkenne-machinelearning-bucket/marian-MT/model')
run.log_artifact(artifact)

wandb: Generating checksum for up to 10000 objects with prefix "marian-MT/model"... Done. 0.0s


<Artifact MT_model>

In [17]:
from google.cloud import storage

def list_bucket_contents(bucket_name, folder_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=folder_name)  # List objects with a given prefix.

    print("Files in {}:".format(folder_name))
    for blob in blobs:
        print(blob.name)

# Provide your bucket name and folder name
bucket_name = 'nkenne-machinelearning-bucket'
folder_name = 'marian-MT/model'
list_bucket_contents(bucket_name, folder_name)

Files in marian-MT/model:
marian-MT/model/runs/
marian-MT/model/runs/Nov06_20-06-25_nkenne-gpu-2/
marian-MT/model/runs/Nov06_20-06-25_nkenne-gpu-2/events.out.tfevents.1699301231.nkenne-gpu-2.387547.3
marian-MT/model/runs/Nov06_20-06-25_nkenne-gpu-2/events.out.tfevents.1699303928.nkenne-gpu-2.387547.4
marian-MT/model/runs/Nov06_20-52-53_nkenne-gpu-2/
marian-MT/model/runs/Nov06_20-52-53_nkenne-gpu-2/events.out.tfevents.1699303984.nkenne-gpu-2.387547.5
marian-MT/model/runs/Nov06_20-56-10_nkenne-gpu-2/
marian-MT/model/runs/Nov06_20-56-10_nkenne-gpu-2/events.out.tfevents.1699304170.nkenne-gpu-2.387547.6
marian-MT/model/runs/Nov07_14-51-10_nkenne-gpu-2/
marian-MT/model/runs/Nov07_14-51-10_nkenne-gpu-2/events.out.tfevents.1699368671.nkenne-gpu-2.387547.7
marian-MT/model/runs/Nov07_20-31-15_nkenne-gpu-2/
marian-MT/model/runs/Nov07_20-31-15_nkenne-gpu-2/events.out.tfevents.1699389076.nkenne-gpu-2.387547.8
marian-MT/model/runs/Nov08_10-34-39_nkenne-gpu-2/
marian-MT/model/runs/Nov08_10-34-39_nken

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [11]:
!wandb --help

Usage: wandb [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  agent         Run the W&B agent
  artifact      Commands for interacting with artifacts
  controller    Run the W&B local sweep controller
  disabled      Disable W&B.
  docker        Run your code in a docker container.
  docker-run    Wrap `docker run` and adds WANDB_API_KEY and WANDB_DOCKER...
  enabled       Enable W&B.
  import        Commands for importing data from other systems
  init          Configure a directory with Weights & Biases
  job           Commands for managing and viewing W&B jobs
  launch        Launch or queue a W&B Job.
  launch-agent  Run a W&B launch agent.
  launch-sweep  Run a W&B launch sweep (Experimental).
  login         Login to Weights & Biases
  offline       Disable W&B sync
  online        Enable W&B sync
  pull          Pull files from Weights & Biases
  restore       Restore code, config and docker state

In [20]:
import wandb
artifact = wandb.Artifact('MT_model', type='model')
artifact

<Artifact MT_model>

In [25]:
!wandb verify

Default host selected: https://api.wandb.ai
Find detailed logs for this test at: /var/tmp/tmprmtvm79i/wandb
❌
Cannot run wandb verify against api.wandb.ai


In [32]:
import wandb

# Always initialize a W&B run to start tracking
wandb.init()

# Download your Model Version files
path = wandb.use_artifact("artifacts/model-yjojiz8c:v0").download()


Problem at: /var/tmp/ipykernel_645993/2659448742.py 4 <module>


KeyboardInterrupt: 

In [37]:
checkpoint_model = MarianMTModel.from_pretrained("artifacts/model-zjwwud3z:v0")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint_model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(56836, 512, padding_idx=56835)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(56836, 512, padding_idx=56835)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,),

In [38]:
import nltk.translate.meteor_score as meteor

# Read your Yoruba test data
with open("small_vocab_yor", "r") as f:
    test_data_yor = f.readlines()

# Translate the Yoruba sentences and store them as hypotheses
hypotheses = []
for line in test_data_yor:
    input_ids = tokenizer(line, return_tensors="pt")["input_ids"].to(device)
    translated_text = checkpoint_model.generate(input_ids)[0]
    translated_text = tokenizer.decode(translated_text, skip_special_tokens=True)
    hypotheses.append(translated_text)

# Read the English references
with open("small_vocab_en", "r") as f:
    references = f.readlines()

In [39]:


# Calculate the METEOR score
meteor_scores = []
for i in range(len(hypotheses)):
    # Tokenize the reference and hypothesis
    reference_tokens = references[i].split()
    hypothesis_tokens = hypotheses[i].split()

    # Calculate METEOR score
    meteor_score = meteor.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score)

# Print the results
for i in range(len(hypotheses)):
    print("Input sentence:", test_data_yor[i])
    print("Predicted translation:", hypotheses[i])
    print("Actual translation:", references[i])
    print(f"METEOR score: {meteor_scores[i]:.4f}")
    print()

# Calculate the overall METEOR score
overall_meteor_score = sum(meteor_scores) / len(meteor_scores)
print(f"Overall METEOR score: {overall_meteor_score:.4f}")

Input sentence: Ó sése kí n rin  ìrìn àjò lọ sí ibi tó farasin ní oṣù January.

Predicted translation: He started me to travel to his affected place in January.
Actual translation: I might travel out to a quiet place in the month of January. .

METEOR score: 0.2715

Input sentence: Mo fẹ́ràn láti rẹjú ní wàrawàra lẹ̀yìn oúnjẹ ò̩sán lojojúmọ́.

Predicted translation: I loved how to take care of it before the day's bread later.
Actual translation: I like to take a nap immediately after lunch everyday.

METEOR score: 0.3064

Input sentence: Mo kórìíra láti máa lo sí àpèjẹ pèlú àwọn ọ̀rẹ́ mi ṣùgbọ́n mo fẹ́ràn láti máa wo tẹlifíṣọ̀n

Predicted translation: I loathed to celebrate with friends but I like to watch TV.
Actual translation: I  don’t like going out to parties with friends but I like watching TV..

METEOR score: 0.5435

Input sentence: Ẹ̀rù bà mí nígbà tí mo rí fọ́tò ẹnì kan tó wọ aṣọ ológun ní ọ̀dẹ̀dẹ̀ ilé náà.

Predicted translation: I was afraid at seeing someo

In [40]:
import nltk.translate.bleu_score as bleu


# Calculate the BLEU score
bleu_scores = []
for i in range(len(hypotheses)):
    # Tokenize the reference and hypothesis
    reference_tokens = references[i].split()
    hypothesis_tokens = hypotheses[i].split()

    # Calculate BLEU score (using the default settings)
    bleu_score = bleu.sentence_bleu([reference_tokens], hypothesis_tokens)
    bleu_scores.append(bleu_score)

# Print the results
for i in range(len(hypotheses)):
    print("Input sentence:", test_data_yor[i])
    print("Predicted translation:", hypotheses[i])
    print("Actual translation:", references[i])
    print(f"BLEU score: {bleu_scores[i]:.4f}")
    print()

# Calculate the overall BLEU score (average of individual scores)
overall_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Overall BLEU score: {overall_bleu_score:.4f}")


Input sentence: Ó sése kí n rin  ìrìn àjò lọ sí ibi tó farasin ní oṣù January.

Predicted translation: He started me to travel to his affected place in January.
Actual translation: I might travel out to a quiet place in the month of January. .

BLEU score: 0.0000

Input sentence: Mo fẹ́ràn láti rẹjú ní wàrawàra lẹ̀yìn oúnjẹ ò̩sán lojojúmọ́.

Predicted translation: I loved how to take care of it before the day's bread later.
Actual translation: I like to take a nap immediately after lunch everyday.

BLEU score: 0.0000

Input sentence: Mo kórìíra láti máa lo sí àpèjẹ pèlú àwọn ọ̀rẹ́ mi ṣùgbọ́n mo fẹ́ràn láti máa wo tẹlifíṣọ̀n

Predicted translation: I loathed to celebrate with friends but I like to watch TV.
Actual translation: I  don’t like going out to parties with friends but I like watching TV..

BLEU score: 0.2919

Input sentence: Ẹ̀rù bà mí nígbà tí mo rí fọ́tò ẹnì kan tó wọ aṣọ ológun ní ọ̀dẹ̀dẹ̀ ilé náà.

Predicted translation: I was afraid at seeing someone who

In [41]:
!gsutil cp -r artifacts gs://nkenne-machinelearning-bucket/marian-MT/model

Copying file://artifacts/model-yjojiz8c:v0/training_args.bin [Content-Type=application/octet-stream]...
Copying file://artifacts/model-yjojiz8c:v0/generation_config.json [Content-Type=application/json]...
Copying file://artifacts/model-yjojiz8c:v0/pytorch_model.bin [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copyi